In [159]:
import pandas as pd
import csv
import re
import warnings
warnings.filterwarnings('ignore')

In [160]:
df = pd.read_csv('private/R_df_to_csv.csv', delimiter='|', encoding='latin-1')

In [161]:
#### stripping HTML tags ####
p = '<.*?>'
for col in df.columns:
    df[col] = df[col].replace(p, '', regex=True)

In [162]:
# splitting categories into main and sub, dropping non-Book main categories #
regex = r"\/(.*?)\/.*"

df['sub_cat'] = df['categoryPath'].str.extract(regex) # sub categories, one level below Books.
df['main_cat'] = df['categoryPath'].astype(str).str[:5] 
df = df[df.main_cat == 'Books'] #dropping non-Book categories

# drop rows missing critical data #
df = df.dropna(axis=0, how='any', subset=['longDn', 'sub_cat','name'])
df.isnull().sum().sort_values(ascending=False)

In [163]:
print("There are " + str(len(df.sub_cat.value_counts())) + " categories of books.")
counts = df['sub_cat'].value_counts()
threshold = 100
print(str(len(counts[counts>threshold])) + " categories have more than " + str(threshold) +" books.")

There are 74 categories of books.
56 categories have more than 100 books.


In [170]:
df.sub_cat.value_counts()

Children's & Kids' Books                    19800
Medical Books                               19218
Education Books                             14001
History Books                               13231
Sports & Outdoor Books                      10042
Law Books                                    9601
Computers & Technology Books                 9312
Political Books                              9140
Self-Help Books                              6977
Crafts & Hobbies Books                       6465
Mathematics Books                            6303
Philosophy Books                             6090
Health & Wellness Books                      5955
Travel Books                                 5797
Language Arts & Disciplines Books            5749
Reference Books                              5648
Foreign Language Study & Reference Books     5547
Teen & Young Adult Books                     5362
Cookbooks, Food & Wine                       5024
Study Aids & Test Prep Books                 3839


### must remove non-null rows: longDn, sub_cat

In [177]:
df.head()

,name,categoryPath,shortDn,longDn,model,catNode,itemId,parentItemId,sub_cat,main_cat
1,Garth Brooks Anthology: The First Five Years (...,Books/Arts & Entertainment Books/Music Books/M...,The first five years were filled with high adv...,Garth Brooks The Anthology,NaN,3920_582321_1318217,56155941.0,56155941.0,Arts & Entertainment Books,Books
2,Special Edition Harry Potter Paperback Box Set,Books/Children's & Kids' Books/Science Fiction...,The perfect gift for collectors and new reader...,The perfect gift for collectors and new reader...,9780545596275,3920_582053_586182,23466151.0,23466151.0,Children's & Kids' Books,Books
3,StrengthsFinder 2.0,Books/Business & Money Books/Development/General,An updated version of the StrengthsFinder prog...,StrengthsFinder 2.0 features the NEW and UPGRA...,NaN,3920_582349_7428141,5512335.0,5512335.0,Business & Money Books,Books
4,Magnolia Table : A Collection of Recipes for G...,"Books/Cookbooks, Food & Wine/Cookbooks for Ent...",Details Coming Soon,Magnolia Table,NaN,3920_3036003_6652905,290970922.0,290970922.0,"Cookbooks, Food & Wine",Books
5,"Oh, the Places You'll Go!",Books/Children's & Kids' Books/Humor Children'...,"Illustrated in full color, Dr. Seuss's 'The Pl...","Dr. Seussâs wonderfully wise Oh, the Places ...",NaN,3920_582053_582573,378996.0,378996.0,Children's & Kids' Books,Books
